# 0. Setup Installs

Install tensorflow and pycocotools

In [ ]:
!pip install tensorflow=="2.*"
!pip install pycocotools

Make folder for SignLanguageDetection

In [3]:
!mkdir 'SignLanguageDetection/'

In folder 'SignLanguageDetection/' clone labelImg for labelling image

In [ ]:
!cd SignLanguageDetection/ && git clone https://github.com/tzutalin/labelImg

In folder 'SignLanguageDetection/' clone tensorflow models for object detection

In [ ]:
!cd SignLanguageDetection/ && git clone https://github.com/tensorflow/models

Installation of the Object Detection API

In [3]:
%%bash
cd SignLanguageDetection/models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash
cd SignLanguageDetection/models/research
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
%%bash
cd SignLanguageDetection/models/research/
python object_detection/builders/model_builder_tf2_test.py

# 1. Setup Paths

In [6]:
import os

In [1]:
WORKSPACE_PATH = 'SignLanguageDetection/workspace'
SCRIPTS_PATH = 'SignLanguageDetection/scripts'
APIMODEL_PATH = 'SignLanguageDetection/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

In [11]:
os.mkdir(WORKSPACE_PATH)
os.mkdir(SCRIPTS_PATH)
os.mkdir(ANNOTATION_PATH)
os.mkdir(IMAGE_PATH)
os.mkdir(MODEL_PATH)
os.mkdir(PRETRAINED_MODEL_PATH)
os.mkdir(IMAGE_PATH+'/train')
os.mkdir(IMAGE_PATH+'/test')

# 2. Create Label Map

In [14]:
labels = [{'name':'A', 'id':1}, {'name':'B', 'id':2}, {'name':'C', 'id':3}, {'name':'D', 'id':4}, {'name':'E', 'id':5},
          {'name':'F', 'id':6}, {'name':'G', 'id':7}, {'name':'H', 'id':8}, {'name':'I', 'id':9}, {'name':'J', 'id':10},
          {'name':'K', 'id':11}, {'name':'L', 'id':12}, {'name':'M', 'id':13}, {'name':'N', 'id':14}, {'name':'O', 'id':15},
          {'name':'P', 'id':16}, {'name':'Q', 'id':17}, {'name':'R', 'id':18}, {'name':'S', 'id':19}, {'name':'T', 'id':20},
          {'name':'U', 'id':21}, {'name':'V', 'id':22}, {'name':'W', 'id':23}, {'name':'X', 'id':24}, {'name':'Y', 'id':25},
          {'name':'Z', 'id':26}]

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records

In [ ]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

# 4. Copy Model Config to Training Folder

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [ ]:
!tar -xzvf "SignLanguageDetection/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz" -C "SignLanguageDetection/workspace/pre-trained-models/"

In [16]:
os.mkdir(CHECKPOINT_PATH)
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

# 5. Update Config For Transfer 

In [3]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [4]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [11]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [12]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [15]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [16]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [17]:
!python {APIMODEL_PATH + '/research/object_detection/model_main_tf2.py'} --model_dir=SignLanguageDetection/workspace/models/my_ssd_mobnet --pipeline_config_path=SignLanguageDetection/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000

2021-06-07 05:19:15.193547: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-06-07 05:19:17.080075: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-06-07 05:19:17.106911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-06-07 05:19:17.107528: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-06-07 05:19:17.107578: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-06-07 05:19:17.111714: I tensorflow/stream_executor/platform/default

# 7. Save and Convert Model

In [ ]:
!python {APIMODEL_PATH + '/research/object_detection/export_tflite_graph_tf2.py'} --pipeline_config_path=SignLanguageDetection/workspace/models/my_ssd_mobnet/pipeline.config --trained_checkpoint_dir=SignLanguageDetection/workspace/models/my_ssd_mobnet/ --output_directory=SignLanguageDetection/workspace/models/

In [19]:
SAVED_MODEL_PATH = MODEL_PATH+'/saved_model'
SAVED_MODEL_TFLITE_PATH = MODEL_PATH+'/saved_tflite_model'

In [20]:
os.mkdir(SAVED_MODEL_TFLITE_PATH)

In [21]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open(SAVED_MODEL_TFLITE_PATH + '/model.tflite', 'wb') as f:
    f.write(tflite_model)

# 8. Load Train Model From Checkpoint

In [5]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [6]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 9. Detect in Real-Time

In [7]:
import cv2 
import numpy as np

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [9]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [10]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break